In [1]:
import os 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

os.chdir('/home/melissa/PROJECT_DIRECTORIES/EEGFeatureExtraction/Scripts/Preprocessing/')
%run sleep_score_gmm.py
%run load_files.py


In [6]:
train_ids = ['S7068',  'S7070', 'S7071', 'S7074']

In [7]:
train_ids_df = []
directory_path = '/home/melissa/PREPROCESSING/SYNGAP1/numpyformat_baseline/'
for animal in train_ids:
    print(animal)
    load_files = LoadFiles(directory_path, animal)
    data_1, brain_state_1 = load_files.load_one_analysis_file(start_times_dict = SYNGAP_baseline_start, end_times_dict = SYNGAP_baseline_end)
    noise_filter_1 = NoiseFilter(data_1, brain_state_file = brain_state_1, channelvariables = channel_variables,ch_type = 'all')
    bandpass_filtered_data_1 = noise_filter_1.filter_data_type()
    eeg_chan = np.split(bandpass_filtered_data_1[3], 17280, axis = 0)
    packet_loss_dir = '/home/melissa/PREPROCESSING/SYNGAP1/cleaned_br_files/'
    file_name = str(animal) + '_BL1.pkl'
    packet_loss_indices = remove_typical_packet_loss(packet_loss_dir, file_name)
    int_slope_dir = '/home/melissa/RESULTS/ICASSP/ALL_EPOCHS/Int_Slope/'
    slope_df = pd.read_csv(int_slope_dir + str(animal) + '_br_1_all_epochs.csv')  
    slope_identified_indices_df = slope_df.loc[slope_df['Slope'] < -8]
    slope_identified_indices = np.unique(np.array(slope_identified_indices_df['Epoch']))
    
    #dictionary of animal ids of interest and corresponding column
    column_dict = {'S7068': 'AG', 'S7070': 'AU', 'S7071': 'BB', 'S7074': 'BW'}
    file_path = '/home/melissa/'
    file_name = 'autoscorer_validation.xlsx'
    vis_score = extract_sleep_stages(file_path, file_name, column_name = column_dict.get(animal))
    sleep_stage_count = []
    pass_indices = [0, 17279, 17280]
    clean_indices = []
    for idx in slope_identified_indices:
        if idx in pass_indices:
            pass
        elif idx + 1 in slope_identified_indices:
            pass
        else:
            prev_idx = idx - 1 
            next_idx = idx + 1
            sleep_stage_count.append(vis_score[prev_idx])
            sleep_stage_count.append(vis_score[next_idx])
    
    for idx in np.arange(0, 17280, 1):
        if idx not in slope_identified_indices:
            clean_indices.append(vis_score[idx])
    
    stage_unique, stage_counts = np.unique(sleep_stage_count, return_counts=True)
    print('noisy epochs')
    print(stage_unique)
    print(stage_counts)
    
    clean_unique, clean_counts = np.unique(clean_indices, return_counts=True)
    print('clean epochs')
    print(clean_unique)
    print(clean_counts)

S7068
noisy epochs
[0 1 2]
[1632  984   78]
clean epochs
[0 1 2]
[6898 6390  324]
S7070
noisy epochs
[0 1 2]
[1669  513  132]
clean epochs
[0 1 2]
[6849 6527  956]
S7071
noisy epochs
[0 1 2]
[1585  268   71]
clean epochs
[0 1 2]
[5970 7254 1197]
S7074
noisy epochs
[0 1 2]
[1682  343   21]
clean epochs
[0 1 2]
[7013 5924  284]
